## Agent ReAct
ReAct to wzorzec łączący Reasoning + Acting: model krokowo rozumuje (planuje), po czym wykonuje akcje narzędziowe (zapytania do API, wyszukiwanie, obliczenia), a otrzymane obserwacje włączane są do kolejnych kroków rozumowania.

### Instalacja bibliotek

In [1]:
!pip install -q langchain langchain-openai python-dotenv

### Przygotowanie modelu

In [2]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
print("Model gotowy.")


Model gotowy.


### Definiowanie narzędzi

In [38]:
from langchain.tools import tool

@tool
def add_numbers(**kwargs:int) -> int:
    """Add numbers together."""
    return sum(kwargs['kwargs'].values())

@tool
def company_info(name: str) -> str:
    "return info about company"
    data = {
        "OpenAI": "Firma zajmująca się sztuczną inteligencją, twórca ChatGPT. Istnieje od 10 lat. ",
        "LangChain": "Framework do budowania aplikacji AI z LLM. Istnieje od 4 lat.",
        "Tesla": "Producent samochodów elektrycznych. Istnieje 23 lata."
    }
    return data.get(name, "Nie mam informacji o tej firmie.")

tools = [add_numbers, company_info]

### Agent ReAct

In [39]:
from langchain.agents import initialize_agent, AgentType
import warnings
warnings.filterwarnings('ignore') #ignoruje zalecenie użycia LangGraph zamiast langchain.agents

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=6
)

### Test agenta

In [40]:
# Zadanie informacyjne + zadanie matematyczne
agent.run("Powiedz mi, czym zajmują się firmy LangChain, Tesla oraz OpenAI. Następnie podaj sumę liczby lat na rynku tych 3 firm.")



> Entering new AgentExecutor chain...
Thought: Najpierw muszę uzyskać informacje o firmach LangChain, Tesla i OpenAI, aby dowiedzieć się, czym się zajmują. Następnie będę musiał ustalić, ile lat każda z tych firm działa na rynku, aby obliczyć ich sumę.

Action:
```
{
  "action": "company_info",
  "action_input": {"name": "LangChain"}
}
```

Observation: Framework do budowania aplikacji AI z LLM. Istnieje od 4 lat.
Thought:Muszę teraz uzyskać informacje o firmie Tesla.

Action:
```
{
  "action": "company_info",
  "action_input": {"name": "Tesla"}
}
```


Observation: Producent samochodów elektrycznych. Istnieje 23 lata.
Thought:Muszę teraz uzyskać informacje o firmie OpenAI, aby zakończyć zbieranie danych o wszystkich trzech firmach.

Action:
```
{
  "action": "company_info",
  "action_input": {"name": "OpenAI"}
}
```


Observation: Firma zajmująca się sztuczną inteligencją, twórca ChatGPT. Istnieje od 10 lat. 
Thought:Teraz mam wszystkie potrzebne informacje o firmach LangChain, Tesl

'LangChain zajmuje się budowaniem aplikacji AI z LLM, Tesla to producent samochodów elektrycznych, a OpenAI to firma zajmująca się sztuczną inteligencją, twórca ChatGPT. Suma lat na rynku tych trzech firm wynosi 37.'